# Value at Risk (VaR) and Stress Testing

This notebook demonstrates how to perform Value at Risk (VaR) calculations and stress testing on a synthetic portfolio using Polars and Python.

In [ ]:
import os
import polars as pl
import numpy as np


## 1. Load Benchmark Data
We will use benchmark returns as the basis for simulating asset returns.

In [ ]:
datasets_dir = os.path.abspath(os.path.join('..', 'datasets'))
benchmarks = pl.read_csv(f"{datasets_dir}/benchmarks.csv")
benchmarks = benchmarks.with_columns([
    pl.col('date').str.strptime(pl.Datetime, format='%Y-%m-%dT%H:%M:%S%.f').alias('date')
])


## 2. Simulate Asset Returns and Portfolio Construction
We create a synthetic portfolio of 3 assets with random weights, correlated to the benchmark.

In [ ]:
np.random.seed(42)  # For reproducibility
portfolio_value = 1_000_000  # $1M portfolio

asset_returns = benchmarks.select([
    pl.col('date'),
    (pl.col('benchmark_return') * 1.2 + np.random.normal(0, 0.001, benchmarks.height)).alias('asset1_return'),
    (pl.col('benchmark_return') * 0.8 + np.random.normal(0, 0.002, benchmarks.height)).alias('asset2_return'),
    (pl.col('benchmark_return') * 0.5 + np.random.normal(0, 0.003, benchmarks.height)).alias('asset3_return')
])

weights = np.array([0.5, 0.3, 0.2])  # 50% in asset1, 30% in asset2, 20% in asset3
portfolio_returns = asset_returns.with_columns([
    (weights[0] * pl.col('asset1_return') +
     weights[1] * pl.col('asset2_return') +
     weights[2] * pl.col('asset3_return')).alias('portfolio_return')
])
portfolio_returns = portfolio_returns.with_columns([
    (pl.col('portfolio_return') * portfolio_value).alias('dollar_change')
])
portfolio_returns.head()


## 3. Value at Risk (VaR) Calculation
We calculate the 1-day VaR at 95% and 99% confidence levels.

In [ ]:
var_95 = portfolio_returns.select([pl.col('dollar_change').quantile(0.05).alias('VaR_95')])[0, 0]
var_99 = portfolio_returns.select([pl.col('dollar_change').quantile(0.01).alias('VaR_99')])[0, 0]
print(f"1-day Value at Risk (95% confidence): ${-var_95:.2f}")
print(f"1-day Value at Risk (99% confidence): ${-var_99:.2f}")


## 4. VaR Attribution by Asset Type
We attribute risk by asset type using groupby aggregation.

In [ ]:
returns = pl.concat([
    portfolio_returns.with_columns([pl.lit('asset1').alias('asset_type'), pl.col('asset1_return').alias('return')]),
    portfolio_returns.with_columns([pl.lit('asset2').alias('asset_type'), pl.col('asset2_return').alias('return')]),
    portfolio_returns.with_columns([pl.lit('asset3').alias('asset_type'), pl.col('asset3_return').alias('return')])
], how='vertical')
var_by_type = returns.group_by('asset_type').agg([
    pl.col('return').std().alias('std_return'),
    pl.col('return').mean().alias('mean_return')
])
print('VaR by Asset Type:')
print(var_by_type)


## 5. Stress Testing
We define and apply several stress scenarios to the portfolio.

In [ ]:
stress_scenarios = [
    ('Market Crash', -0.07),
    ('Tech Bubble', -0.04),
    ('Interest Rate Shock', -0.025),
    ('Normal Volatility', -0.01)
]

stress_results = []
for scenario, shock in stress_scenarios:
    dollar_impact = shock * portfolio_value
    stress_results.append({'scenario': scenario, 'return': shock, 'dollar_impact': dollar_impact})

stress_df = pl.DataFrame(stress_results)
print('Stress Test Results:')
print(stress_df)


## 6. Save Results
We save the portfolio returns and stress test results as CSV files.

In [ ]:
portfolio_returns.write_csv(f'{datasets_dir}/portfolio_returns.csv')
stress_df.write_csv(f'{datasets_dir}/stress_test_results.csv')
print('Saved results.')
